In [7]:
import tensorflow as tf
import numpy as np
import pandas as pd
from statsmodels import api as sm
from scipy import stats
from sklearn.metrics import mean_squared_error
import os
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras import activations
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorboard import notebook

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import sklearn
import scipy.io
import numpy
import cv2

from sklearn.svm import SVC,NuSVC
from sklearn.model_selection import GridSearchCV

In [2]:
root_dir = os.path.abspath(os.getcwd())+"/256_ObjectCategories/"
#add the arrays that we can use for training and testing
main_x = []
main_y = []
for subdir, dirs, files in os.walk(root_dir):
    for dir in dirs:
        current_dir = os.path.join(root_dir) + dir
        after = current_dir.split(".",1)[1]
        for file in os.listdir(current_dir):
            filename = os.fsdecode(file)
            if filename.endswith(".jpg"):
                current_file = cv2.imread(os.path.join(current_dir,filename),0)
                current_file = cv2.resize(current_file, dsize=(54, 140), interpolation=cv2.INTER_CUBIC)
                main_x.append(current_file)
                main_y.append(after)
                continue
            else:
                continue
    break
images_x = np.array(main_x)
images_y = np.array(main_y)
print(np.shape(images_x))

(30607, 140, 54)


In [6]:
temp_x_train, x_test, temp_y_train, y_test = train_test_split(images_x,images_y,test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(temp_x_train,temp_y_train,test_size=0.2,random_state=42)
print(len(x_train))
print(len(x_test))
print(len(x_val))

19588
6122
4897


In [ ]:
#setup the SVM
#grid search cross validation has already been imported ready for hyperparameter tuning
svm = SVC(class_weight='balanced')
#add some stuff for some shitty hyperparameter tuning here. You guys figure out the smarter way to do it. I belive the smarter way is using the range functionality to narrow down the range of values but you will need to investigate
param_grid = [
    {'C':[0.1,1,10,100],'kernel':['linear']},
    {'C':[0.1,1,10,100],'gamma':[0.1,0.01,0.001],'kernel':['rbf']},
    {'C':[0.1,1,10,100],'degree':[3,4,5],'kernel':['poly']},
]
grid_search = GridSearchCV(svm, param_grid)
grid_search.fit(x_train,y_train)
grid_search.cv_results_
best_system = np.argmin(grid_search.cv_results_['rank_test_score'])
params = grid_search.cv_results_['params'][best_system]
print(params)
svm = SVC().set_params(**params)
svm.fit(x_train,y_train)

In [8]:
#build a DCNN real quick
num_epochs = 50

def build_model(num_classes,output_activation=None):
    inputs = keras.Input(shape=(54,140),name='img')

    x = layers.Conv2D(filters=8, kernel_size=(3,3),padding='same',activation='relu')(inputs)
    x = layers.Conv2D(filters=8,kernel_size=(3,3),padding='same',activation=None)(x)

    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.SpatialDropout2D(0.2)(x)

    x = layers.MaxPool2D(pool_size=(2,2))(x)

    x = layers.Conv2D(filters=8, kernel_size=(3,3),padding='same',activation='relu')(x)
    x = layers.Conv2D(filters=8,kernel_size=(3,3),padding='same',activation=None)(x)

    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.SpatialDropout2D(0.2)(x)

    x = layers.MaxPool2D(pool_size=(2,2))(x)

    x = layers.Conv2D(filters=8, kernel_size=(3,3),padding='same',activation='relu')(x)
    x = layers.Conv2D(filters=8,kernel_size=(3,3),padding='same',activation=None)(x)

    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.SpatialDropout2D(0.2)(x)

    x = layers.Flatten()(x)

    x = layers.Dense(256,activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(64,activation='relu')(x)
    outputs = layers.Dense(num_classes,activation=output_activation)(x)

    model_cnn = keras.Model(inputs=inputs, outputs=outputs, name='CNN')

    return model_cnn

model_cnn = build_model(256)
model_cnn.summary()

In [ ]:
model_cnn.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
optimizer=keras.optimizers.Adam(),
metrics=['accuracy'])
history = model_cnn.fit(x_train,y_train,
batch_size=128,
epochs=num_epochs,
validation_data=(x_test,y_test))